In [3]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.model_selection import train_test_split
from sklearn.model_selection import StratifiedKFold
from sklearn.model_selection import cross_val_score, cross_validate
from sklearn import svm
from sklearn.metrics import *

from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.svm import SVC
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score, precision_score, recall_score, f1_score, roc_auc_score



## Step-by-Step Guide

**Step 1: Import Libraries and Extract Files**
- First, import the necessary libraries and extract the files from the ZIP archive. Additionally, list the contents of the extraction directory to confirm the files are present.
- If the extracted files are still in zip format, we need to unzip these individual CSV files as well. 

**os.listdir:** Lists the files in the extraction directory to confirm they were extracted correctly.

In [5]:
import os

print("Current working directory:", os.getcwd())


Current working directory: C:\Users\avary


In [8]:
import os

# List files in the current working directory
print("Files in current directory:", os.listdir(os.getcwd()))


Files in current directory: ['.anaconda', '.cisco', '.conda', '.condarc', '.continuum', '.ipynb_checkpoints', '.ipython', '.jupyter', '.matplotlib', '.ms-ad', '.virtual_documents', '3D Objects', 'anaconda3', 'AppData', 'Application Data', 'Bosch Project', 'bosch-production-line-performance.zip', 'bosch_data', 'Box', 'Char - Conceptual.csv', 'Char - Procedural.csv', 'Contacts', 'Cookies', 'Creative Cloud Files', 'Cronbach leave-one item-out.ipynb', 'cronbach_alpha_results.csv', 'cronbach_alpha_results.xlsx', 'Desktop', 'Documents', 'Downloads', 'Exp_Survey.xlsx', 'Favorites', 'Homoscedasticity.ipynb', 'Krippendorff_Alpha.xlsx', 'Links', 'Local Settings', 'MicrosoftEdgeBackups', 'Music', 'My Documents', 'NetHood', 'Normality_Cheking.py', 'NTUSER.DAT', 'ntuser.dat.LOG1', 'ntuser.dat.LOG2', 'NTUSER.DAT{578b3544-d63b-11eb-8d8f-8c7d9e0940b4}.TxR.0.regtrans-ms', 'NTUSER.DAT{578b3544-d63b-11eb-8d8f-8c7d9e0940b4}.TxR.1.regtrans-ms', 'NTUSER.DAT{578b3544-d63b-11eb-8d8f-8c7d9e0940b4}.TxR.2.regtra

In [10]:
import zipfile
import os

# Define the path to the zip file
zip_file_path = 'bosch-production-line-performance.zip'

# Check if the file exists and is not empty
if not os.path.exists(zip_file_path) or os.path.getsize(zip_file_path) == 0:
    print("The zip file does not exist or is empty. Please check the file.")
else:
    try:
        # Define the directory to extract files into
        extraction_dir = 'bosch_data'

        # Extract the zip file
        with zipfile.ZipFile(zip_file_path, 'r') as zip_ref:
            zip_ref.extractall(extraction_dir)

        # List the extracted files
        extracted_files = os.listdir(extraction_dir)
        print("Extracted files:", extracted_files)
    except zipfile.BadZipFile:
        print("Error: The file is not a valid zip file or is corrupted.")
    except Exception as e:
        print(f"An unexpected error occurred: {e}")


Extracted files: ['.ipynb_checkpoints', 'sample_submission.csv', 'sample_submission.csv.zip', 'test_categorical.csv', 'test_categorical.csv.zip', 'test_date.csv', 'test_date.csv.zip', 'test_numeric.csv', 'test_numeric.csv.zip', 'train_categorical.csv', 'train_categorical.csv.zip', 'train_date.csv', 'train_date.csv.zip', 'train_numeric.csv', 'train_numeric.csv.zip']


In [12]:

# Extract individual CSV files from their zip archives in bosch-production-line-performance.zip file
for file_name in extracted_files:
    if file_name.endswith('.zip'):
        with zipfile.ZipFile(os.path.join(extraction_dir, file_name), 'r') as zip_ref:
            zip_ref.extractall(extraction_dir)

# List files again after nested extraction
extracted_files = os.listdir(extraction_dir)
print("Files after nested extraction:", extracted_files)

# Print contents of the extraction directory
for root, dirs, files in os.walk(extraction_dir):
    for name in files:
        print(os.path.join(root, name))

Files after nested extraction: ['.ipynb_checkpoints', 'sample_submission.csv', 'sample_submission.csv.zip', 'test_categorical.csv', 'test_categorical.csv.zip', 'test_date.csv', 'test_date.csv.zip', 'test_numeric.csv', 'test_numeric.csv.zip', 'train_categorical.csv', 'train_categorical.csv.zip', 'train_date.csv', 'train_date.csv.zip', 'train_numeric.csv', 'train_numeric.csv.zip']
bosch_data\sample_submission.csv
bosch_data\sample_submission.csv.zip
bosch_data\test_categorical.csv
bosch_data\test_categorical.csv.zip
bosch_data\test_date.csv
bosch_data\test_date.csv.zip
bosch_data\test_numeric.csv
bosch_data\test_numeric.csv.zip
bosch_data\train_categorical.csv
bosch_data\train_categorical.csv.zip
bosch_data\train_date.csv
bosch_data\train_date.csv.zip
bosch_data\train_numeric.csv
bosch_data\train_numeric.csv.zip


In [14]:
# Define the paths to the extracted CSV files
train_numeric_path = os.path.join(extraction_dir, 'train_numeric.csv')
train_categorical_path = os.path.join(extraction_dir, 'train_categorical.csv')
train_date_path = os.path.join(extraction_dir, 'train_date.csv')

# Check if files exist and print the paths
print("train_numeric_path:", train_numeric_path)
print("train_categorical_path:", train_categorical_path)
print("train_date_path:", train_date_path)
print("train_numeric_path exists:", os.path.exists(train_numeric_path))
print("train_categorical_path exists:", os.path.exists(train_categorical_path))
print("train_date_path exists:", os.path.exists(train_date_path))


# Load the first 5 rows of the numeric features
train_numeric_features = pd.read_csv(train_numeric_path, nrows=5)
print("Train Numeric Features Head (first 5 rows):")
print(train_numeric_features.head(10))

# Print the number of numeric features
print(f"\nNumber of train numeric features: {len(train_numeric_features.columns)}")
print(f"Train Numeric feature names: {train_numeric_features.columns.tolist()}")

# Print summary statistics for the numeric features
print("\nSummary statistics for train numeric features:")
print(train_numeric_features.describe())

# Load the first 5 rows of the categorical features
train_categorical_features = pd.read_csv(train_categorical_path, nrows=5)
print("\nTrain Categorical Features Head (first 5 rows):")
print(train_categorical_features.head(5))

# Print the number of categorical features
print(f"\nNumber of train categorical features: {len(train_categorical_features.columns)}")
print(f"Train Categorical feature names: {train_categorical_features.columns.tolist()}")

# Print summary statistics for the categorical features
print("\nSummary statistics for train categorical features:")
print(train_categorical_features.describe())


train_numeric_path: bosch_data\train_numeric.csv
train_categorical_path: bosch_data\train_categorical.csv
train_date_path: bosch_data\train_date.csv
train_numeric_path exists: True
train_categorical_path exists: True
train_date_path exists: True
Train Numeric Features Head (first 5 rows):
   Id  L0_S0_F0  L0_S0_F2  L0_S0_F4  L0_S0_F6  L0_S0_F8  L0_S0_F10  L0_S0_F12  \
0   4     0.030    -0.034    -0.197    -0.179     0.118      0.116     -0.015   
1   6       NaN       NaN       NaN       NaN       NaN        NaN        NaN   
2   7     0.088     0.086     0.003    -0.052     0.161      0.025     -0.015   
3   9    -0.036    -0.064     0.294     0.330     0.074      0.161      0.022   
4  11    -0.055    -0.086     0.294     0.330     0.118      0.025      0.030   

   L0_S0_F14  L0_S0_F16  ...  L3_S50_F4245  L3_S50_F4247  L3_S50_F4249  \
0     -0.032      0.020  ...           NaN           NaN           NaN   
1        NaN        NaN  ...           NaN           NaN           NaN   
2

## Difference Between Inner and Left Joins

**Inner Join:

- Definition: Returns only the rows where there is a match in both datasets.
- Effect: If an Id is missing in any dataset, the corresponding rows will be excluded from the result.
- Usage: Useful when you want to ensure that only complete data points across all datasets are included.

**Left Join:

- Definition: Returns all rows from the left dataset, and the matched rows from the right dataset. If no match is found, NaN values are filled in for columns from the right dataset.
- Effect: Keeps all rows from the left dataset, regardless of whether there is a matching row in the right dataset.
- Usage: Useful when you want to retain all rows from the primary dataset (left) and add information from the secondary dataset (right) when available.

## 1. Data Preparation in Training Set: Sequence for handling missing values, balancing dataset, and applying PCA for feature reduction

**1.1. Handle Missing Values:** First, address any missing values in dataset. This ensures that the dataset is complete and prevents                                        issues during the balancing and PCA steps.
   
   - Given the kernel keeps dying, we can implement a more memory-efficient approach by processing each chunk individually and not storing all processed chunks in memory simultaneously. We can achieve this by saving intermediate results to disk.
       
       - ***Step-by-Step Implementation***
         
         - ***1.1.1 Identify Columns to Drop:*** First, iterate over each file to identify columns with more than 40% missing values across                                                 all chunks.
         
         - ***1.1.2. Process Each Chunk Separately:*** Drop the identified columns, fill remaining missing values with zero, and save the 
                                                     processed chunks to disk.
              
               -  1.1.2.1. Given the context of the Bosch production-line-performance dataset, filling missing values with zero can be 
                           appropriate for several reasons
                            - 1.1.2.1.1. Since missing values indicate the absence of a measurement, which can logically be represented by                                          zero.
                            - 1.1.2.1.2. Consistency Across Feature Types:Since the dataset is divided into numerical, categorical, and                                              date features, applying a consistent strategy (e.g., filling with zero) can simplify the                                                    preprocessing steps and ensure uniformity.
                                       
         - ***1.1.3. Check and Remove Duplicates Before Merging:*** Verify if there are any duplicate Ids in each dataset and remove them if 
                                                                  they exist.                                       
       
         - ***1.1.4. Merge Processed Data:*** Load the processed chunks from disk and merge them using the Id column with an inner join to 
                                            ensure only complete data points are included (an Id is missing in any dataset, the 
                                            corresponding rows will be excluded from the result)..

**1.2. Balance the Classes:** After handling missing values, balance the classes. This step ensures that the dataset is not biased towards                               the majority class, which is particularly important before dimensionality reduction.
    
                             **Balancing Techniques:**
                                  - **Oversampling:** Use SMOTE to generate synthetic samples for the minority class.
                                  - **Undersampling:** Randomly sample from the majority class to reduce its size.
                                         - ***In this project, Undersampling is used to balance the dataset

**1.3. Convert Categorical Variables to Dummy Variables:** After balancing the classes, there are some non-numeric values (categoricl                                                                  features) in dataset that are causing issues with the StandardScaler in PCA.                                                                These non-numeric values need to be handled before standardizing the features.                                                              one-hot encoding will be used to handle non-numeric (categorical) features. This                                                            will convert categorical variables into a series of binary (dummy) variables.

**1.4. Apply PCA:** Finally, apply PCA to reduce the dimensionality of the dataset. This step should come after balancing the classes to                       ensure that PCA captures the variability in the balanced dataset.


 - **After handling missing values, the remaining missing values per column are shown as"Missing values after handling:"

In [17]:
#1.1.1 Identify Columns to Drop

import pandas as pd
import os

# Define paths
train_numeric_path = os.path.join('bosch_data', 'train_numeric.csv')
train_categorical_path = os.path.join('bosch_data', 'train_categorical.csv')
## train_date_path = os.path.join('bosch_data', 'train_date.csv')

# Function to collect missing values per column in chunks
def collect_missing_values(file_path, chunk_size=1000):
    missing_counts = None
    total_rows = 0
    chunk_counter = 0

    for chunk in pd.read_csv(file_path, chunksize=chunk_size, engine='python'):
        chunk_counter += 1
        print(f"Processing chunk {chunk_counter}...")
        if missing_counts is None:
            missing_counts = chunk.isnull().sum()
        else:
            missing_counts += chunk.isnull().sum()
        total_rows += len(chunk)

    missing_proportion = missing_counts / total_rows
    return missing_proportion

# Collect missing values proportions for each file
print("Collecting missing values for numeric data...")
numeric_missing_values = collect_missing_values(train_numeric_path)
print("Collecting missing values for categorical data...")
categorical_missing_values = collect_missing_values(train_categorical_path)
#print("Collecting missing values for date data...")
#date_missing_values = collect_missing_values(train_date_path)

# Identify columns to drop (more than 40% missing values)
columns_to_drop_numeric = numeric_missing_values[numeric_missing_values > 0.4].index.tolist()
columns_to_drop_categorical = categorical_missing_values[categorical_missing_values > 0.4].index.tolist()
#columns_to_drop_date = date_missing_values[date_missing_values > 0.4].index.tolist()

print(f"Columns to drop in numeric data: {columns_to_drop_numeric}")
print(f"Columns to drop in categorical data: {columns_to_drop_categorical}")
#print(f"Columns to drop in date data: {columns_to_drop_date}")


Processing chunk 1...
Processing chunk 2...
Processing chunk 3...
Processing chunk 4...
Processing chunk 5...
Processing chunk 6...
Processing chunk 7...
Processing chunk 8...
Processing chunk 9...
Processing chunk 10...
Processing chunk 11...
Processing chunk 12...
Processing chunk 13...
Processing chunk 14...
Processing chunk 15...
Processing chunk 16...
Processing chunk 17...
Processing chunk 18...
Processing chunk 19...
Processing chunk 20...
Processing chunk 21...
Processing chunk 22...
Processing chunk 23...
Processing chunk 24...
Processing chunk 25...
Processing chunk 26...
Processing chunk 27...
Processing chunk 28...
Processing chunk 29...
Processing chunk 30...
Processing chunk 31...
Processing chunk 32...
Processing chunk 33...
Processing chunk 34...
Processing chunk 35...
Processing chunk 36...
Processing chunk 37...
Processing chunk 38...
Processing chunk 39...
Processing chunk 40...
Processing chunk 41...
Processing chunk 42...
Processing chunk 43...
Processing chunk 44.

In [19]:

#1.1.2. Process Each Chunk Separately

# Function to process each chunk and save to disk
def process_and_save_chunks(file_path, columns_to_drop, output_file, chunk_size=1000):
    if os.path.exists(output_file):
        os.remove(output_file)
    
    chunk_counter = 0
    
    for chunk in pd.read_csv(file_path, chunksize=chunk_size, engine='python'):
        chunk_counter += 1
        print(f"Processing chunk {chunk_counter}...")

        # Drop columns with more than 40% missing values
        chunk.drop(columns=columns_to_drop, inplace=True)
        
        # Fill remaining missing values with zero
        chunk.fillna(0, inplace=True)
        
        # Save each processed chunk to disk
        chunk.to_csv(output_file, mode='a', header=not os.path.exists(output_file), index=False)
        print(f"Chunk {chunk_counter} processed and saved.")

# Process numeric data
process_and_save_chunks(train_numeric_path, columns_to_drop_numeric, 'train_numeric_processed.csv')

# Process categorical data
process_and_save_chunks(train_categorical_path, columns_to_drop_categorical, 'train_categorical_processed.csv')

# Process date data
#process_and_save_chunks(train_date_path, columns_to_drop_date, 'train_date_processed.csv')


Processing chunk 1...
Chunk 1 processed and saved.
Processing chunk 2...
Chunk 2 processed and saved.
Processing chunk 3...
Chunk 3 processed and saved.
Processing chunk 4...
Chunk 4 processed and saved.
Processing chunk 5...
Chunk 5 processed and saved.
Processing chunk 6...
Chunk 6 processed and saved.
Processing chunk 7...
Chunk 7 processed and saved.
Processing chunk 8...
Chunk 8 processed and saved.
Processing chunk 9...
Chunk 9 processed and saved.
Processing chunk 10...
Chunk 10 processed and saved.
Processing chunk 11...
Chunk 11 processed and saved.
Processing chunk 12...
Chunk 12 processed and saved.
Processing chunk 13...
Chunk 13 processed and saved.
Processing chunk 14...
Chunk 14 processed and saved.
Processing chunk 15...
Chunk 15 processed and saved.
Processing chunk 16...
Chunk 16 processed and saved.
Processing chunk 17...
Chunk 17 processed and saved.
Processing chunk 18...
Chunk 18 processed and saved.
Processing chunk 19...
Chunk 19 processed and saved.
Processing 

In [21]:

## 1.1.3. Check and Remove Duplicates Before Merging

# Load processed data
train_numeric_processed = pd.read_csv('train_numeric_processed.csv')
train_categorical_processed = pd.read_csv('train_categorical_processed.csv')
train_date_processed = pd.read_csv('train_date_processed.csv')

# Check for duplicates in each dataset
print("Checking for duplicates in numeric data...")
numeric_duplicates = train_numeric_processed[train_numeric_processed.duplicated(subset='Id', keep=False)]
print(f"Number of duplicate Ids in numeric data: {numeric_duplicates.shape[0]}")

print("Checking for duplicates in categorical data...")
categorical_duplicates = train_categorical_processed[train_categorical_processed.duplicated(subset='Id', keep=False)]
print(f"Number of duplicate Ids in categorical data: {categorical_duplicates.shape[0]}")

#print("Checking for duplicates in date data...")
#date_duplicates = train_date_processed[train_date_processed.duplicated(subset='Id', keep=False)]
#print(f"Number of duplicate Ids in date data: {date_duplicates.shape[0]}")

# Remove duplicates if they exist
train_numeric_processed = train_numeric_processed.drop_duplicates(subset='Id')
train_categorical_processed = train_categorical_processed.drop_duplicates(subset='Id')
#train_date_processed = train_date_processed.drop_duplicates(subset='Id')


Checking for duplicates in numeric data...
Number of duplicate Ids in numeric data: 0
Checking for duplicates in categorical data...
Number of duplicate Ids in categorical data: 0


In [23]:
#1.1.4. Merge Processed Data

# Perform inner joins to avoid duplications
train_merged = train_numeric_processed.merge(train_categorical_processed, on='Id', how='inner')
#train_merged = train_merged.merge(train_date_processed, on='Id', how='inner')

# Show class distribution before balancing
print("\nClass Distribution Before Balancing:")
print(train_merged['Response'].value_counts())



Class Distribution Before Balancing:
Response
0    1176868
1       6879
Name: count, dtype: int64


## 1.2. Balance the Classes:

To balance the dataset by downsampling the majority class (class 0) to match the size of the minority class (class 1) and process the data in chunks, we will follow these steps:

- **1.2.1. Balance Each Chunk:**
   - The 'Response' column is likely only present in the numeric file. We should only balance the numeric data and not the categorical          data.
        - Process the data in chunks to avoid loading the entire dataset into memory.
        - For each chunk, separate the data by class (class 0 and class 1).
        - Downsample the majority class (class 0) to match the size of the minority class (class 1).
        - Combine the balanced classes and shuffle the data.
        - Save the balanced chunk to disk.

- **1.2.2. Combine and Shuffle:**
         - Load the balanced chunks from disk and combine them into a single DataFrame.
         - Shuffle the combined balanced dataset to ensure randomness.

  **1.2.3. Save to New File:**
        - Save the final balanced and shuffled dataset to a new CSV file.
        - Save the balanced dataset to a new Excel file with two sheets: one for the balanced data and one for the original balanced data.
          - This approach ensures that the dataset is balanced without running into memory issues and preserves the original balanced data             for future reference.

In [29]:
##1.2.1. Balance Each Chunk

import pandas as pd
from sklearn.utils import resample
import os

# Define paths
train_numeric_path = 'train_numeric_processed.csv'
train_categorical_path = 'train_categorical_processed.csv'

# Define the output file for balanced data
balanced_output_file = 'train_numeric_balanced.csv'

# Remove the output file if it exists (to start fresh)
if os.path.exists(balanced_output_file):
    os.remove(balanced_output_file)

# Function to balance and save chunks
def balance_and_save_chunks(file_path, output_file, chunk_size=1000):
    chunk_counter = 0

    for chunk in pd.read_csv(file_path, chunksize=chunk_size, engine='python'):
        chunk_counter += 1
        print(f"Processing chunk {chunk_counter}...")

        # Separate majority and minority classes
        if 'Response' in chunk.columns:
            class_0 = chunk[chunk['Response'] == 0]
            class_1 = chunk[chunk['Response'] == 1]

            if len(class_1) > 0:
                # Downsample majority class
                class_0_downsampled = resample(class_0,
                                               replace=False,  # sample without replacement
                                               n_samples=len(class_1),  # match minority class size
                                               random_state=42)  # reproducible results

                # Combine minority class with downsampled majority class
                balanced_chunk = pd.concat([class_0_downsampled, class_1])
            else:
                balanced_chunk = chunk

            # Shuffle the balanced chunk
            balanced_chunk = balanced_chunk.sample(frac=1, random_state=42).reset_index(drop=True)

            # Save each balanced chunk to disk
            balanced_chunk.to_csv(output_file, mode='a', header=not os.path.exists(output_file), index=False)
            print(f"Chunk {chunk_counter} balanced and saved.")
        else:
            print(f"Chunk {chunk_counter} skipped (no 'Response' column).")

# Balance numeric data
balance_and_save_chunks(train_numeric_path, balanced_output_file)



Processing chunk 1...
Chunk 1 balanced and saved.
Processing chunk 2...
Chunk 2 balanced and saved.
Processing chunk 3...
Chunk 3 balanced and saved.
Processing chunk 4...
Chunk 4 balanced and saved.
Processing chunk 5...
Chunk 5 balanced and saved.
Processing chunk 6...
Chunk 6 balanced and saved.
Processing chunk 7...
Chunk 7 balanced and saved.
Processing chunk 8...
Chunk 8 balanced and saved.
Processing chunk 9...
Chunk 9 balanced and saved.
Processing chunk 10...
Chunk 10 balanced and saved.
Processing chunk 11...
Chunk 11 balanced and saved.
Processing chunk 12...
Chunk 12 balanced and saved.
Processing chunk 13...
Chunk 13 balanced and saved.
Processing chunk 14...
Chunk 14 balanced and saved.
Processing chunk 15...
Chunk 15 balanced and saved.
Processing chunk 16...
Chunk 16 balanced and saved.
Processing chunk 17...
Chunk 17 balanced and saved.
Processing chunk 18...
Chunk 18 balanced and saved.
Processing chunk 19...
Chunk 19 balanced and saved.
Processing chunk 20...
Chunk 2

In [31]:
##1.2.2. Combine and Shuffle & 1.2.3. Save to New File

# Load processed and balanced numeric data
balanced_numeric = pd.read_csv('train_numeric_balanced.csv')

# Load processed categorical data
balanced_categorical = pd.read_csv(train_categorical_path)

# Perform inner joins to avoid duplications
train_merged = balanced_numeric.merge(balanced_categorical, on='Id', how='inner')

# Show class distribution after balancing
print("\nClass Distribution After Balancing:")
print(train_merged['Response'].value_counts())

# Save the merged data to disk
train_merged.to_csv('train_merged.csv', index=False)




Class Distribution After Balancing:
Response
0    12879
1     6879
Name: count, dtype: int64


## A revision to the *1.2.1. Balance Each Chunk* to ensure the classes are balanced in PCA-transformed dataset:

Since the balancing step (1.2.1. Balance Each Chunk) didn't perfectly balance the classes: 
  - Class Distribution After Balancing: Response    0    12879
                                                    1     6879
                                                    Number of data points after PCA: 19758
 I address the downsampling part first, ensuring we correctly balance the class distribution.

- **Load the PCA-transformed dataset:** The Id column is not a feature for the model and should be excluded before PCA and balancing.
- **Downsample the majority class (class 0) to match the size of the minority class (class 1).
- **Combine the downsampled majority class with the minority class. 

In [61]:
# A revision to the *1.2.1. Balance Each Chunk* to balance the classes in PCA-transformed dataset

import pandas as pd
from sklearn.utils import resample

# Load the merged data
train_merged_path = 'train_merged.csv'
train_merged = pd.read_csv(train_merged_path)

# Show initial class distribution after merging
print("Initial Class Distribution After Merging:")
print(train_merged['Response'].value_counts())

# Separate majority and minority classes
class_0 = train_merged[train_merged['Response'] == 0]
class_1 = train_merged[train_merged['Response'] == 1]

# Downsample majority class (class 0)
class_0_downsampled = resample(class_0,
                               replace=False,  # sample without replacement
                               n_samples=len(class_1),  # match minority class size
                               random_state=42)  # reproducible results

# Combine minority class with downsampled majority class
balanced_data = pd.concat([class_0_downsampled, class_1])

# Shuffle the balanced dataset
balanced_data = balanced_data.sample(frac=1, random_state=42).reset_index(drop=True)

# Show new class distribution after downsampling
print("New Class Distribution After Downsampling:")
print(balanced_data['Response'].value_counts())

# Save the balanced dataset to disk
balanced_data.to_csv('train_balanced.csv', index=False)

# Load the balanced dataset and print its columns
train_balanced = pd.read_csv('train_balanced.csv')
print("Columns in train_balanced.csv:", train_balanced.columns)


Initial Class Distribution After Merging:
Response
0    12879
1     6879
Name: count, dtype: int64
New Class Distribution After Downsampling:
Response
1    6879
0    6879
Name: count, dtype: int64
Columns in train_balanced.csv: Index(['Id', 'L3_S29_F3315', 'L3_S29_F3318', 'L3_S29_F3321', 'L3_S29_F3324',
       'L3_S29_F3327', 'L3_S29_F3330', 'L3_S29_F3333', 'L3_S29_F3336',
       'L3_S29_F3339',
       ...
       'L3_S29_F3466', 'L3_S29_F3469', 'L3_S29_F3472', 'L3_S29_F3475',
       'L3_S29_F3478', 'L3_S29_F3481', 'L3_S29_F3484', 'L3_S29_F3487',
       'L3_S29_F3490', 'L3_S29_F3493'],
      dtype='object', length=204)


## 1.3. Convert Categorical Variables to Dummy Variables

- **Load the Merged and Balanced Data:** Load the balanced dataset from the CSV file.
- **Identify Categorical Columns:** Identify columns with object type (categorical) using select_dtypes(include=['object'])..
- **Apply One-Hot Encoding:** Convert categorical variables to binary/dummy variables using pd.get_dummies()  while preserving all other                                 columns, including Id and Response.
- **Save the Data:** Save the dataset with dummy variables to a new CSV file.


In [63]:
# 1.3. Convert Categorical Variables to Dummy Variables

import pandas as pd
import os

# Load the merged and balanced data
merged_data_path = 'train_balanced.csv'
train_merged = pd.read_csv(merged_data_path)

# Identify categorical columns
categorical_columns = train_merged.select_dtypes(include=['object']).columns.tolist()
print(f"Categorical columns: {categorical_columns}")

# Apply one-hot encoding to categorical columns
train_with_dummies = pd.get_dummies(train_merged, columns=categorical_columns)

# Save the data with dummy variables to disk
dummy_output_file = 'train_with_dummies.csv'
train_with_dummies.to_csv(dummy_output_file, index=False)

print(f"Categorical variables converted to dummy variables and saved to {dummy_output_file}.")



Categorical columns: ['L3_S29_F3317', 'L3_S29_F3320', 'L3_S29_F3323', 'L3_S29_F3326', 'L3_S29_F3329', 'L3_S29_F3332', 'L3_S29_F3335', 'L3_S29_F3338', 'L3_S29_F3341', 'L3_S29_F3344', 'L3_S29_F3347', 'L3_S29_F3350', 'L3_S29_F3353', 'L3_S29_F3356', 'L3_S29_F3359', 'L3_S29_F3362', 'L3_S29_F3364', 'L3_S29_F3366', 'L3_S29_F3369', 'L3_S29_F3372', 'L3_S29_F3375', 'L3_S29_F3378', 'L3_S29_F3381', 'L3_S29_F3384', 'L3_S29_F3387', 'L3_S29_F3390', 'L3_S29_F3392', 'L3_S29_F3394', 'L3_S29_F3397', 'L3_S29_F3400', 'L3_S29_F3403', 'L3_S29_F3406', 'L3_S29_F3409', 'L3_S29_F3411', 'L3_S29_F3414', 'L3_S29_F3416', 'L3_S29_F3418', 'L3_S29_F3420', 'L3_S29_F3423', 'L3_S29_F3426', 'L3_S29_F3429', 'L3_S29_F3432', 'L3_S29_F3435', 'L3_S29_F3438', 'L3_S29_F3441', 'L3_S29_F3444', 'L3_S29_F3446', 'L3_S29_F3448', 'L3_S29_F3451', 'L3_S29_F3454', 'L3_S29_F3457', 'L3_S29_F3460', 'L3_S29_F3463', 'L3_S29_F3466', 'L3_S29_F3469', 'L3_S29_F3472', 'L3_S29_F3475', 'L3_S29_F3478', 'L3_S29_F3481', 'L3_S29_F3484', 'L3_S29_F3487', 'L

## 1.4. Apply PCA

- **4.1. Load the Data with Dummy Variables:** Load the processed dataset with dummy variables from disk.
- **4.2. Separate Features and Target:** Exclude the Id and Response columns.
- **4.3. Standardize the Features:** Standardize the features using StandardScaler.
     - This step ensures that each feature has a mean of 0 and a standard deviation of 1. Standardization is crucial for PCA because it is         sensitive to the variances of the initial variables.
     
- **4.4. Apply PCA:** Perform PCA to retain 95% of the variance.
     - The PCA model is initialized with n_components=0.95. This parameter means that PCA will select the number of principal components          needed to explain 95% of the variance in the data and then projects the original features onto these components.
     
- **4.5. Create a DataFrame with PCA Components:** Create a DataFrame for PCA components and add Id and Response columns.
     - The fit_transform method is called on the standardized features X_scaled. This method ***first fits the PCA model to the data,              identifying the directions (principal components) that maximize variance. Then, it transforms the data into these principal                components, effectively reducing the dimensionality.

- **4.6. Save the PCA-Transformed Data:** Save the PCA-transformed data to a new CSV file.
- **4.7. Display Resulting Dimensions:** Print the number of features and data points after PCA

In [117]:
# 1.4. Apply PCA

import pandas as pd
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA
import joblib

# Load the data with dummy variables
dummy_data_path = 'train_with_dummies.csv'
train_with_dummies = pd.read_csv(dummy_data_path)

# Separate features and target
X = train_with_dummies.drop(columns=['Id', 'Response'])
y = train_with_dummies['Response']

# Standardize the features
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

# Apply PCA
pca = PCA(n_components=0.95)  # Retain 95% of the variance
X_pca = pca.fit_transform(X_scaled)

# Save the scaler and PCA model for Using in Test dataset Jupyter Notebook
joblib.dump(scaler, 'scaler.pkl')
joblib.dump(pca, 'pca_model.pkl')

# Combine PCA components with Id and Response columns
pca_columns = [f'PC{i+1}' for i in range(X_pca.shape[1])]
train_pca = pd.DataFrame(X_pca, columns=pca_columns)
train_pca['Id'] = train_with_dummies['Id']
train_pca['Response'] = y

# Save the PCA-transformed data to disk
pca_output_file = 'train_pca.csv'
train_pca.to_csv(pca_output_file, index=False)

# Display resulting dimensions
print(f"PCA applied and data saved to {pca_output_file}.")
print(f"Number of PCA components: {len(pca_columns)}")
print(f"Number of data points: {train_pca.shape[0]}")

# Display first few rows of the transformed dataset
print("First few rows of the PCA-transformed dataset:")
print(train_pca.head())

PCA applied and data saved to train_pca.csv.
Number of PCA components: 49
Number of data points: 13758
First few rows of the PCA-transformed dataset:
         PC1       PC2       PC3       PC4       PC5       PC6       PC7  \
0  -8.186405 -0.396205  4.250472 -1.835672 -1.547754 -2.151279  4.463917   
1  -8.062441  0.156391 -0.386765  0.528704 -5.683512 -0.556741  1.329681   
2  -8.171390 -0.974851 -5.077047  1.975859  1.477286  0.451297  3.646403   
3  -7.978406 -0.136337  4.273147  0.567075 -3.065830 -1.456736  4.337433   
4  15.583505 -1.419875  1.277697  2.402119 -4.032090  0.666433  0.260718   

        PC8       PC9      PC10  ...      PC42      PC43      PC44      PC45  \
0  2.511987 -0.701525 -1.121637  ... -0.146751  0.115463  0.226702 -0.629758   
1 -0.288601  1.051572 -1.118955  ...  0.075148  0.114235 -0.724175  0.254171   
2  0.740817  0.026243 -0.525844  ... -0.204018 -0.398804 -0.199776  0.066643   
3  2.405708  5.103199  0.988584  ...  0.149829  0.102835 -0.204230 -0.098

## 2. Data Splitting and Model Training

**2.1. Data Splitting:** Split the balanced and PCA-transformed dataset into training and testing sets.

**2.2. Initial Model Selection (Using Cross-Validation):** Implement Logistic Regression, Decision Tree, and SVM, Random Forest, Gradient                                                              Boosting, with cross-validation to evaluate their performance.
   - Start by implementing several models with default parameters.
   - Use cross-validation to compare the performance of these models.
   - Select the best-performing models based on cross-validation results.

**2.3. Hyperparameter Tuning:** Perform hyperparameter tuning using Grid Search or Random Search for the best-performing models.

   - For the best-performing models from the previous step, do hyperparameter tuning to optimize their performance.
   - Use Grid Search or Randomized Search with cross-validation to find the best set of hyperparameters.
   
   
**2.4. Final Model Training and Testing:** Train the final model ***using the entire training set(instead different folds of training                                                 sets)*** with the best hyperparameters found in step 2.3 and evaluate it on the test set.

   - 2.4.1. **Train the Final Model: Use the entire training dataset with the best hyperparameters found during the tuning process.***
        
   - 2.4.2. **Evaluate the Final Model on Test Set:** Assess the model's performance on the held-out test set using the same metrics.
        
        - Test Set with response variable: Evaluate the final model with the best hyperparameters using the evaluation metrics                                                         (Accuracy, Precision, Recall, F1-Score, and ROC-AUC) 
       
       - ***Test Set without  response variable: Similar to this project, test datset does not contain the actual 'Response' values, since                                                   they are  not available in the test set for prediction), I could not be able to compute                                                    the evaluation metrics like accuracy, precision, recall, F1-score, and ROC AUC directly.                                                    Instead, I could only save the predictions and probabilities without calculating the                                                        evaluation metrics:

In [84]:
## 2.1. Data Splitting: 80% training, 20% testing

from sklearn.model_selection import train_test_split

# Load the PCA-transformed data
pca_data_path = 'train_pca.csv'
train_pca = pd.read_csv(pca_data_path)

# Separate features and target
X = train_pca.drop(columns=['Id', 'Response'])
y = train_pca['Response']

# Split the data into training and testing sets (80% training, 20% testing)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, stratify=y, random_state=42)

print("Training set size:", X_train.shape)
print("Test set size:", X_test.shape)


Training set size: (11006, 49)
Test set size: (2752, 49)


## 2.2. Model Selection and Cross-Validation Notes

# Cross-validation results for the three models are as follows:

**Logistic Regression Scores:**
  - Accuracy: 0.5907 ± 0.0089
  - Precision: 0.5941 ± 0.0093
  - Recall: 0.5726 ± 0.0116
  - F1-Score: 0.5831 ± 0.0096
  - ROC-AUC: 0.6117 ± 0.0140

**Decision Tree Scores:**
  - Accuracy: 0.5725 ± 0.0101
  - Precision: 0.5790 ± 0.0138
  - Recall: 0.5357 ± 0.0366
  - F1-Score: 0.5556 ± 0.0181
  - ROC-AUC: 0.5947 ± 0.0124

**SVM Scores:*
  - Accuracy: 0.5944 ± 0.0092
  - Precision: 0.5956 ± 0.0097
  - Recall: 0.5886 ± 0.0113
  - F1-Score: 0.5920 ± 0.0093
  - ROC-AUC: 0.6083 ± 0.0110
  
**Random Forest Scores**
  - Accuracy: 0.5976 ± 0.0089
  - Precision: 0.6065 ± 0.0118
  - Recall: 0.5566 ± 0.0050
  - F1 Score: 0.5804 ± 0.0058
  - ROC AUC: 0.5976 ± 0.0089

**Gradient Boosting Scores**
  - Accuracy: 0.5887 ± 0.0049
  - Precision: 0.5922 ± 0.0059
  - Recall: 0.5699 ± 0.0081
  - F1 Score: 0.5808 ± 0.0047
  - ROC AUC: 0.5887 ± 0.0049  
  
**Analysis:**
- **SVM** generally performs the best across most metrics, especially in accuracy, precision, recall, and F1-score, though Logistic Regression is close in terms of ROC-AUC.

- **Logistic Regression** also performs well but slightly lags behind SVM in most metrics.

- **Decision Tree** has the lowest performance among the three models, particularly in recall and F1-score, indicating that it might not generalize as well as the other two models.

# Selected Model:
Based on the performance metrics, **SVM and Random Forest** appear to have the best performance overall, especially considering accuracy and ROC AUC scores. Below are the next steps for hyperparameter tuning for these two models.


In [86]:
## 2.2. Initial Model Selection (Using Cross-Validation with Comprehensive Metrics)

## SVM, Decsion Tree, Logistic Regression

from sklearn.model_selection import cross_val_score, cross_validate
from sklearn.metrics import make_scorer, precision_score, recall_score, f1_score, roc_auc_score

# Define the scoring metrics
scoring = {
    'accuracy': 'accuracy',
    'precision': make_scorer(precision_score),
    'recall': make_scorer(recall_score),
    'f1': make_scorer(f1_score),
    'roc_auc': 'roc_auc'
}

# Logistic Regression
log_reg = LogisticRegression(C=10, solver='liblinear', max_iter=1000, random_state=42)
scores_log_reg = cross_validate(log_reg, X, y, cv=5, scoring=scoring)

# Decision Tree
decision_tree = DecisionTreeClassifier(max_depth=5, min_samples_leaf=1, min_samples_split=2, random_state=42)
scores_decision_tree = cross_validate(decision_tree, X, y, cv=5, scoring=scoring)

# SVM
svm = SVC(C=1, gamma='scale', kernel='linear', random_state=42)
scores_svm = cross_validate(svm, X, y, cv=5, scoring=scoring)

# Print the scores
print("Logistic Regression Scores:")
for metric in scoring.keys():
    print(f"{metric}: {scores_log_reg['test_' + metric].mean()} ± {scores_log_reg['test_' + metric].std()}")

print("\nDecision Tree Scores:")
for metric in scoring.keys():
    print(f"{metric}: {scores_decision_tree['test_' + metric].mean()} ± {scores_decision_tree['test_' + metric].std()}")

print("\nSVM Scores:")
for metric in scoring.keys():
    print(f"{metric}: {scores_svm['test_' + metric].mean()} ± {scores_svm['test_' + metric].std()}")


Logistic Regression Scores:
accuracy: 0.590709641525703 ± 0.00889770297331064
precision: 0.5941380616618354 ± 0.009342852206887468
recall: 0.5726126849894292 ± 0.011567550917343527
f1: 0.5831430010510197 ± 0.009572305072478816
roc_auc: 0.6117040522518314 ± 0.013961855195871483

Decision Tree Scores:
accuracy: 0.5725388706432334 ± 0.010050438337239665
precision: 0.5789865699437662 ± 0.013760604257205605
recall: 0.535696511627907 ± 0.03662884188544757
f1: 0.5556123002244837 ± 0.01810247384450608
roc_auc: 0.5946515155228257 ± 0.01238535596285684

SVM Scores:
accuracy: 0.5944167765632793 ± 0.009239517437164387
precision: 0.5955851297931378 ± 0.009746085737277994
recall: 0.5886038054968287 ± 0.011275728964449241
f1: 0.5920311307881839 ± 0.009262300112691885
roc_auc: 0.6082512974611584 ± 0.010993408250832202


In [90]:
## 2.2. (Continued.) Initial Model Selection (Using Cross-Validation with Comprehensive Metrics)

## random Forest & Gradient Boosting

from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier
from sklearn.model_selection import cross_validate
from sklearn.metrics import make_scorer, accuracy_score, precision_score, recall_score, f1_score, roc_auc_score

# Initialize models
models = {
    'Random Forest': RandomForestClassifier(random_state=42),
    'Gradient Boosting': GradientBoostingClassifier(random_state=42)
}

# Define scoring metrics
scoring = {
    'accuracy': make_scorer(accuracy_score),
    'precision': make_scorer(precision_score),
    'recall': make_scorer(recall_score),
    'f1': make_scorer(f1_score),
    'roc_auc': make_scorer(roc_auc_score)
}

# Evaluate each model using cross-validation with multiple metrics
for model_name, model in models.items():
    print(f"Evaluating {model_name}...")
    scores = cross_validate(model, X_train, y_train, cv=5, scoring=scoring)
    
    accuracy = scores['test_accuracy']
    precision = scores['test_precision']
    recall = scores['test_recall']
    f1 = scores['test_f1']
    roc_auc = scores['test_roc_auc']
    
    print(f"Accuracy: {accuracy.mean():.4f} ± {accuracy.std():.4f}")
    print(f"Precision: {precision.mean():.4f} ± {precision.std():.4f}")
    print(f"Recall: {recall.mean():.4f} ± {recall.std():.4f}")
    print(f"F1 Score: {f1.mean():.4f} ± {f1.std():.4f}")
    print(f"ROC AUC: {roc_auc.mean():.4f} ± {roc_auc.std():.4f}")
    print("\n")



Evaluating Random Forest...
Accuracy: 0.5976 ± 0.0089
Precision: 0.6065 ± 0.0118
Recall: 0.5566 ± 0.0050
F1 Score: 0.5804 ± 0.0058
ROC AUC: 0.5976 ± 0.0089


Evaluating Gradient Boosting...
Accuracy: 0.5887 ± 0.0049
Precision: 0.5922 ± 0.0059
Recall: 0.5699 ± 0.0081
F1 Score: 0.5808 ± 0.0047
ROC AUC: 0.5887 ± 0.0049




## 2.3. Hyperparameter Tuning Notes

## Hyper Parameter Tunning Results for SVM & Random Forest of the best-performing models in step 2.3

**Best parameters for SVM using GridSearch**
  - Best parameters for SVM:", {'C': 1, 'gamma': 'scale', 'kernel': 'rbf'}
  - Best cross-validation score for SVM:", 0.5977641966513032
    - Inconsistent cross-validation folds between initial model selection (5-fold) and hyperparameter tuning (3-fold) (Use different number       of folds for two stages in SVM) due to limitation in computational resources

**Best parameters for Random Forest using randomized Search**
  - Best parameters for Random Forest: {'n_estimators': 500, 'min_samples_split': 10, 'min_samples_leaf': 2, 'max_depth': None,                                                      'bootstrap': True}
   - Best cross-validation score for Random Forest: 0.6079409450167355

## Select Model:
- Let's proceed to train and evaluate the final model using the entire training dataset with the best hyperparameters obtained from **Grid   Search for SVM and Randomized Search for RandomForest.**
    - I considered it is possible to use different methods for hyperparameter tuning (Randomized Search or Grid Search) for different             models (here SVM & Random Forest) and compare their results. 


In [97]:
## 2.3. Hyperparameter Tuning: (for SVM)

from sklearn.model_selection import GridSearchCV
from sklearn.svm import SVC

# Define parameter grid for SVM
param_grid_svm = {
    'C': [0.1, 1, 10, 100],
    'kernel': ['linear', 'rbf'],
    'gamma': ['scale', 'auto']
}

# Hyperparameter tuning with Grid Search for SVM
# Using verbose=3 will print detailed progress information, which helps in understanding if the process is making progress or getting stuck.
grid_search_svm = GridSearchCV(SVC(random_state=42), param_grid_svm, cv=3, scoring='accuracy', n_jobs=-1, verbose=3)
grid_search_svm.fit(X_train, y_train)


print("Best parameters for SVM:", grid_search_svm.best_params_)
print("Best cross-validation score for SVM:", grid_search_svm.best_score_)


Fitting 3 folds for each of 16 candidates, totalling 48 fits
Best parameters for SVM: {'C': 1, 'gamma': 'scale', 'kernel': 'rbf'}
Best cross-validation score for SVM: 0.5977641966513032


In [99]:
## 2.3. Hyperparameter Tuning: (for Random Forest)

from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import RandomizedSearchCV

# Define parameter grid for Random Forest
param_grid_rf = {
    'n_estimators': [100, 200, 500],
    'max_depth': [5, 10, 20, None],
    'min_samples_split': [2, 5, 10],
    'min_samples_leaf': [1, 2, 5],
    'bootstrap': [True, False]
}

# Hyperparameter tuning with Randomized Search for Random Forest
random_search_rf = RandomizedSearchCV(RandomForestClassifier(random_state=42), param_grid_rf, n_iter=50, cv=5, scoring='accuracy', random_state=42)
random_search_rf.fit(X_train, y_train)

print("Best parameters for Random Forest:", random_search_rf.best_params_)
print("Best cross-validation score for Random Forest:", random_search_rf.best_score_)


Best parameters for Random Forest: {'n_estimators': 500, 'min_samples_split': 10, 'min_samples_leaf': 2, 'max_depth': None, 'bootstrap': True}
Best cross-validation score for Random Forest: 0.6079409450167355


## 2.4.1. Train the Final Model Notes

 - I trained the SVM model using the best hyperparameters found from Grid Search on the entire training dataset.
 - I trained the Random Forest model using the best hyperparameters found from Randomized Search on the entire training dataset.
 - To evaluate models on the sampled test set, I need to use the trained models that I saved after hyperparameter tuning to disk first.        Then, I load them back when I need to perform evaluation.
 - If you do not have the actual test 'Response' values:
 - You can only save the predictions and probabilities without calculating the evaluation metrics:
 
 

In [166]:
## 2.4.1. Train the Final Model with SVM

import pandas as pd
from sklearn.svm import SVC
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, roc_auc_score
import joblib

# Load the PCA-transformed training data
train_pca_path = 'train_pca.csv'
train_pca = pd.read_csv(train_pca_path)

# Separate features and target
X_train_final = train_pca.drop(columns=['Id', 'Response'])
y_train_final = train_pca['Response']

# Best hyperparameters for SVM with probability=True
best_params_svm = {'C': 1, 'gamma': 'scale', 'kernel': 'rbf', 'probability': True, 'random_state': 42}

# Train the final SVM model
final_svm_model = SVC(**best_params_svm)
final_svm_model.fit(X_train_final, y_train_final)

print("Final SVM model trained.")

# Save the SVM model to disk
joblib.dump(final_svm_model, 'final_svm_model.pkl')
print("SVM model saved to disk.")



Final SVM model trained.
SVM model saved to disk.


In [168]:
## 2.4.1. Train the Final Model with Random Forest

from sklearn.ensemble import RandomForestClassifier
import joblib
import pandas as pd

# Load the PCA-transformed training data
train_pca_path = 'train_pca.csv'
train_pca = pd.read_csv(train_pca_path)

# Separate features and target
X_train_final = train_pca.drop(columns=['Id', 'Response'])
y_train_final = train_pca['Response']

# Best hyperparameters for Random Forest
best_params_rf = {
    'n_estimators': 500,
    'min_samples_split': 10,
    'min_samples_leaf': 2,
    'max_depth': None,
    'bootstrap': True
}

# Train the final Random Forest model
final_rf_model = RandomForestClassifier(**best_params_rf, random_state=42)
final_rf_model.fit(X_train_final, y_train_final)

print("Final Random Forest model trained.")

# Save the Random Forest model to disk
joblib.dump(final_rf_model, 'final_rf_model.pkl')
print("Random Forest model saved to disk.")


Final Random Forest model trained.
Random Forest model saved to disk.


# Sample the Test Set for Final Model Evaluation 

**To reduce the number of data points in the test set to 20% of the training dataset size, I randomly got sample from the test dataset. Given that the training dataset has 13,758 data points, 20% of this would be approximately 2,752 data points.

In [170]:
import pandas as pd

# Load the PCA-transformed test data
test_pca_path = 'test_pca.csv'
test_pca = pd.read_csv(test_pca_path)

# Calculate the number of samples needed (20% of the training data size)
num_samples = int(0.2 * 13758)

# Randomly sample the required number of data points from the test set
test_sampled = test_pca.sample(n=num_samples, random_state=42)

# Save the sampled test set to a new CSV file
test_sampled_path = 'test_sampled.csv'
test_sampled.to_csv(test_sampled_path, index=False)

print(f"Sampled {num_samples} data points from the test set and saved to {test_sampled_path}")


Sampled 2751 data points from the test set and saved to test_sampled.csv


## 2.4.2. Evaluate on Test Set Notes

**Id:** This column uniquely identifies each part in the dataset.

**Predicted_Response_SVM:** This column indicates whether the SVM model predicts **the part will fail (1) or not fail (0) quality                                       control***.
**Predicted_Probability_SVM:** This column gives the SVM model's confidence in its prediction, represented as a probability. Higher values                                indicate higher confidence in predicting failure.The predicted probability of the positive class (1) by the                                SVM model for each data point.
  
  - **Example:** 0.425247 (indicating a 42.52% chance that the part will fail quality control)

**Predicted_Response_RF:** This column indicates whether the Random Forest model predicts the part will fail (1) or not fail (0) quality                               control.
**Predicted_Probability_RF:** This column gives the Random Forest model's confidence in its prediction, represented as a probability.                                     Higher values indicate higher confidence in predicting failure.The predicted probability of the positive                                   class (1) by the Random Forest model for each data point.
  
  - **Example:** 0.396531 (indicating a 39.65% chance that the part will fail quality control)

In [177]:
## 2.4.2 Evaluate the Final Models (SVM & random Forest Models) on the Sampled Test Set without Response Value


import pandas as pd
import joblib

# Load the PCA-transformed test data
test_sampled_path = 'test_sampled.csv'
test_sampled = pd.read_csv(test_sampled_path)

# Separate features and Id
X_test_final = test_sampled.drop(columns=['Id'])
test_ids = test_sampled['Id']

# Load the trained models from disk
final_svm_model = joblib.load('final_svm_model.pkl')
final_rf_model = joblib.load('final_rf_model.pkl')

# Predict with SVM
y_test_pred_svm = final_svm_model.predict(X_test_final)
y_test_proba_svm = final_svm_model.predict_proba(X_test_final)[:, 1]  # Probability of the positive class

# Predict with Random Forest
y_test_pred_rf = final_rf_model.predict(X_test_final)
y_test_proba_rf = final_rf_model.predict_proba(X_test_final)[:, 1]  # Probability of the positive class

# Save the predictions to a CSV file
predictions_svm = pd.DataFrame({
    'Id': test_ids,
    'Predicted_Response_SVM': y_test_pred_svm,
    'Predicted_Probability_SVM': y_test_proba_svm
})

predictions_rf = pd.DataFrame({
    'Id': test_ids,
    'Predicted_Response_RF': y_test_pred_rf,
    'Predicted_Probability_RF': y_test_proba_rf
})

predictions_svm.to_csv('svm_predictions.csv', index=False)
predictions_rf.to_csv('rf_predictions.csv', index=False)

print("Predictions saved to svm_predictions.csv and rf_predictions.csv.")

print("Predictions saved to svm_predictions.csv and rf_predictions.csv.")

# Display the first few rows of the prediction DataFrames
print("First few rows of SVM predictions:")
print(predictions_svm.head())

print("\nFirst few rows of Random Forest predictions:")
print(predictions_rf.head())


Predictions saved to svm_predictions.csv and rf_predictions.csv.
Predictions saved to svm_predictions.csv and rf_predictions.csv.
First few rows of SVM predictions:
        Id  Predicted_Response_SVM  Predicted_Probability_SVM
0   201057                       0                   0.425247
1  2272009                       1                   0.678237
2    86999                       0                   0.411028
3   931306                       0                   0.395679
4  1836203                       1                   0.599667

First few rows of Random Forest predictions:
        Id  Predicted_Response_RF  Predicted_Probability_RF
0   201057                      0                  0.396531
1  2272009                      1                  0.629560
2    86999                      0                  0.376309
3   931306                      0                  0.485937
4  1836203                      1                  0.643855
